In [30]:
import os
import sys
parent_dir = os.path.join(os.getcwd(), '..')
if parent_dir not in sys.path: sys.path.append(parent_dir)
from utility.data_utils import *
from utility.visuals import *
from model.dimensional_reduction import *
from model.gans import *
from model.rf import *
from model.dnn import *
from model.knn import *
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split

In [31]:
load_dotenv()
DATA_DIR = os.getenv("DATA_DIR")
PCA_N_COMPONENTS = float(os.getenv("PCA_N_COMPONENTS"))
NOISE_DIM = int(os.getenv("NOISE_DIM"))
BATCH_SIZE = int(os.getenv("BATCH_SIZE"))
NUM_EPOCHS = int(os.getenv("NUM_EPOCHS"))
NUM_EPOCHS = 100

In [32]:
files = get_files(DATA_DIR)
file = files[0]
city_name = file.split('\\')[-1].split('.csv')[0].split('_')[0]

path_name = process_and_transform_data(file, resolution=.5, overwrite=True)
with open(path_name, 'rb') as f:
    result_dict = pickle.load(f)
labels = result_dict['labels']
encoder = result_dict['encoder']
MAX_LEN = result_dict['max length']
file_name = result_dict['file name']
WIDTH = result_dict['width']
HEIGHT = result_dict['height']
unique_labels = [u for u in labels if np.array(np.where(u != 0)).T.shape[0] > 1]
train_labels, test_labels = train_test_split(np.expand_dims(np.array(unique_labels), axis=1), test_size=.2)
pca = PCATransformer(n_components=PCA_N_COMPONENTS)
transformed = pca.fit_and_predict(
    train_labels.reshape(train_labels.shape[0], -1)
)
knn = KNNFeaturesExtractor()
knn.fit(transformed)
features = knn.predict(transformed)
features = features.reshape(len(features), -1)

original dataset size: 238
dataset size with duplicates removed: 172


In [33]:
OUTPUT_DIM = transformed.shape[1]
CONDITION_DIM = features.shape[-1]
x, y, _, _ = get_x_y(test_labels, MAX_LEN=MAX_LEN, encoder=encoder)
x = np.array(x)
y = np.array(y)
test_pca = PCATransformer(n_components=transformed.shape[-1])
test_transformed = test_pca.fit_and_predict(
    x.reshape(x.shape[0], -1)
)
test_features = knn.predict(test_transformed)
test_features = test_features.reshape(len(test_features), -1)

In [34]:
dnn_manager = DNNManager(
    input_dim=CONDITION_DIM, 
    output_dim=OUTPUT_DIM, 
    batch_size=BATCH_SIZE, 
    num_epochs=NUM_EPOCHS,
    lr=3e-4,
    betas=(.5, 0.999)
)
dnn_manager.train(features, transformed, verbose=1)

100%|██████████| 100/100 [00:04<00:00, 22.11it/s]


In [35]:
gan_manager = GANManager(
    noise_dim=NOISE_DIM,
    condition_dim=features.shape[-1],
    output_dim=transformed.shape[1],
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    lr=0.0002,
    betas = (.5, 0.999),
)
gan_manager.train(features, transformed, verbose=1)


100%|██████████| 100/100 [00:12<00:00,  8.01it/s]


In [36]:

rf_manager = RandomForestManager()
rf_manager.train(features, transformed)

Train MSE: 0.8983


In [37]:
knn_manager = KNNManager(n_neighbors=1)
knn_manager.train(features, transformed)

KNN model trained successfully.


In [38]:
gan_output = gan_manager.predict(test_features)
gan_output_shaped = test_pca.predict(gan_output, inverse=True)\
    .reshape(len(gan_output), x.shape[-2], x.shape[-1])
gan_output_shaped[gan_output_shaped < 0] = 0


dnn_output = dnn_manager.predict(test_features)
dnn_output_shaped = test_pca.predict(dnn_output, inverse=True)\
    .reshape(len(dnn_output), x.shape[-2], x.shape[-1])
dnn_output_shaped[dnn_output_shaped < 0] = 0

rf_output = rf_manager.predict(test_features)
rf_output_shaped = test_pca.predict(rf_output, inverse=True)\
    .reshape(len(rf_output), x.shape[-2], x.shape[-1])
rf_output_shaped[rf_output_shaped < 0] = 0



knn_output = knn_manager.predict(test_features)
knn_output_shaped = test_pca.predict(knn_output, inverse=True)\
    .reshape(len(knn_output), x.shape[-2], x.shape[-1])
knn_output_shaped[knn_output_shaped < 0] = 0

In [39]:
knn_output_shaped

array([[[0.09860832, 0.18833384, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.13060536, 0.27611851, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0. 